# YOLOv1

<img src = "https://github.com/fned49/blog/blob/main/YOLO/YOLO Loss/figure/figure1.png?raw=true">

기존 R-CNN 계열의 모델이 classification, localization task에 맞게 서로 다른 loss function을 사용했던 것과 달리,  
YOLO v1 모델은 regression 시 주로 사용되는 SSE(Sum of Squared Error)를 사용합니다.  
위의 그림에서 볼 수 있듯이 Localization loss, Confidence loss, Classification loss의 합으로 구성되어 있습니다. 

#### Localization loss

$$\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(x_i-\hat{x}_i\right)^2 + \left(y_i-\hat{y}_i\right)^2\right]$$  
$$+\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(\sqrt{w_i}-\sqrt{\hat{w}_i}\right)^2 + \left(\sqrt{h_i}-\sqrt{\hat{h}_i}\right)^2\right]$$ 

- $\lambda_{coord}$ : localization loss에 가중치를 주는 파라미터입니다. (=5)
- $S^2$ : grid cell의 총 개수 (=7x7=49)
- $B$ : 각 grid cell이 예측하는 bounding box의 수 (=2)
- $\mathcal{1}_{ij}^{obj}$ : $i$번째 grid cell에서 예측한 $j$번째 bounding box가 실제로 객체를 포함하는지 여부를 나타내는 이진 값입니다. (grid cell에서는 B개의 bounding box를 예측하지만, 그 중 confidence score가 가장 높은 1개의 bounding box만을 학습에 사용합니다.)
- $x_i$, $y_i$, $w_i$, $h_i$ : ground truth box의 $x$, $y$ 좌표와 width, height 크기가 큰 bounding box의 small error가 크기가 작은 bounding box의 error보다 덜 중요하다는 것을 반영하기 위해 $w_i$, $h_i$ 값에 squre root를 씌어주게 됩니다. 
- $\hat{x}_i$, $\hat{y}_i$, $\hat{w}_i$, $\hat{h}_i$ : 예측 bounding box의 $x, y$ 좌표, width, height

#### Confidence loss

$$+\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left(C_i-\hat{C}_i\right)^2$$  
$$+\lambda_{noobj}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{noobj}\left(C_i-\hat{C}_i\right)^2$$  

- $\mathcal{1}_{ij}^{obj}$ : $i$번째 grid cell에서 예측한 $j$번째 bounding box가 실제로 객체를 포함하는지 여부를 나타내는 이진 값입니다.
- $\lambda_{noobj}$ : 객체가 존재하지 않는 셀의 confidence loss에 가중치를 주는 파라미터 입니다. (=0.5)
- $C_i$ : $i$번째 grid cell에 실제로 객체가 포함되어 있는지 여부를 나타내는 이진 값입니다.
- $\hat{C}_i$ : $i$번째 grid cell에서 모델이 예측한 confidence score로, 0에서 1 사이의 값입니다.
- $\mathcal{1}_{ij}^{noobj}$ : $i$번째 grid cell에서 예측한 $j$번째 bounding box가 실제로 객체를 포함하지 않는지 여부를 나타내는 이진 값입니다.


#### Classification loss

$$+\sum_{i=0}^{S^2}\mathcal{1}_{i}^{obj}\sum_{c\in classes}\left(p_i\left(c\right)-\hat{p}_i\left(c\right)\right)$$  

- $p_i\left(c\right)$ : $i$번째 grid cell에서 실제로 클래스 $c$인지 여부를 나타내는 이진 값입니다.
- $\hat{p}_i\left(c\right)$ : 모델이 예측한 $i$번째 grid cell에서 클래스 $c$의 확률을 나타냅니다. 이 값은 0에서 1 사이의 범위에 있습니다.

---

# YOLOv2

<img src = "https://github.com/fned49/blog/blob/main/YOLO/YOLO Loss/figure/figure4.png?raw=true">

논문의 저자는 grid cell에 상대적인 위치 좌표를 예측하는 방법을 선택했습니다. 이는 예측하는 bounding box의 좌표는 0~1 사이의 값을 가짐을 의미합니다. 위의 그림에서 $c_x$, $c_y$는 grid cell의 좌상단 offset입니다. bounding box regression을 통해 얻은 $t_x$, $t_y$ 값에 logistic regression 함수(σ)를 적용하여 0~1 사이의 값을 가지도록 조정했습니다. 

t : 모델의 예측 offset  
p : prior(k - means clustering) -> anchor box size   
c : grid cell의 좌상단 offset    
b : 예측한 bounding box

<img src = "https://github.com/fned49/blog/blob/main/YOLO/YOLO Loss/figure/figure3.png?raw=true">



<img src = "https://github.com/fned49/blog/blob/main/YOLO/YOLO Loss/figure/figure2.png?raw=true">

YOLO v1과 같이 Localization loss, Confidence loss, Classification loss로 구성되어 있고,  
또한 v1과 같이 전체 loss가 SSE(Sum of Squared Error)입니다.

이미지 내 특정 객체의 중심이 특정 grid cell에 있는 경우, 해당 grid cell은 객체를 예측하도록 할당(responsible for)받습니다.

#### Localization loss

$$\lambda^{coord}_{obj}\sum_{i}^{S^2}\sum_{j}^{B}\mathcal{1}_{ij}^{responsible\_obj}\left(x_{ij}^{pred}-x_{ij}^{obj}\right)^2 + \left(y_{ij}^{pred}-y_{ij}^{obj}\right)^2 + \left(w_{ij}^{pred}-w_{ij}^{obj}\right)^2 + \left(h_{ij}^{pred}-h_{ij}^{obj}\right)^2$$

- $\lambda^{coord}_{obj}$ : 객체가 실제로 존재하는 grid cell에 가중치를 부여합니다. 이 값이 높을수록 모델은 bounding box의 위치 정보에 더 큰 가중치를 두어 정확한 위치를 학습하려고 노력하게 됩니다. 반면에 이 값이 낮으면 위치 정보에 대한 영향이 상대적으로 낮아져서 모델이 좀 더 대략적인 위치만을 학습할 수 있게 됩니다. (=5)
- $\sum_{i}^{S^2}$ : grid cell 수 (=13x13=169)
- $\sum_{j}^{B}$ : 각 grid cell의 anchor box 수 (=5)
- $\mathcal{1}_{ij}^{responsible\_obj}$ : $i$번째 그리드 셀의 $j$번째 anchor box가 해당 그리드 셀에서 실제 객체를 담당할 경우 1, 그렇지 않은 경우 0
- $x_{ij}^{pred}$, $y_{ij}^{pred}$, $w_{ij}^{pred}$, $h_{ij}^{pred}$ : 모델이 예측한 bounding box의 중심 좌표와 너비, 높이입니다.
- $x_{ij}^{obj}$, $y_{ij}^{obj}$, $w_{ij}^{obj}$, $h_{ij}^{obj}$ : 실제 객체의 중심 좌표와 너비, 높이입니다.

$$+ \lambda^{coord}_{noobj}\sum_{i}^{S^2}\sum_{j}^{B}\mathcal{1}_{ij}^{no\_responsible\_obj}\left(x_{ij}^{pred}-x_{ij}^{anchor\_center}\right)^2 + \left(y_{ij}^{pred}-y_{ij}^{anchor\_center}\right)^2 + \left(w_{ij}^{pred}-w_{ij}^{anchor\_default}\right)^2 + \left(h_{ij}^{pred}-h_{ij}^{anchor\_default}\right)^2$$  

- $\lambda^{coord}_{noobj}$: no-object bounding box에 대한 가중치(이미지 내에서 객체를 예측하도록 할당받은(anchor box가 있는) grid cell의 수는 매우 많지만, 실제 객체의 중심을 anchor box에 포함한 grid cell에 비해 중요도가 떨어집니다. 이를 조정하기 위해 not responsible for한 grid cell에 대한 가중치값입니다.)
- $\mathcal{1}_{ij}^{no\_responsible\_obj}$ : $i$번째 grid cell의 $j$번째 anchor box가 해당 그리드 셀에서 실제 객체를 담당하지 않을 경우 1, 담당 할 경우 0
- $x_{ij}^{pred}$, $y_{ij}^{pred}$, $w_{ij}^{pred}$, $h_{ij}^{pred}$ : 모델이 예측한 bounding box의 중심 좌표와 너비, 높이입니다.

- $x_{ij}^{anchor\_center}$, $y_{ij}^{anchor\_center}$: 모델이 예측한 anchor box의 위치에 대한 학습 목표입니다. 각 grid cell에서 예측한 anchor box의 중심 좌표를 나타냅니다. 이 값들은 0.5로 지정됩니다. 그 이유는, 예측한 anchor box를 해당 그리드 셀 내의 중심에 위치시키기 위함입니다.
- $w_{ij}^{anchor\_default}$, $h_{ij}^{anchor\_default}$: 예측한 anchor box의 크기의 학습 목표로, grid cell 전체 크기가 되도록 1로 지정합니다. 이는 모델이 예측한 anchor box의 너비와 높이가 grid cell 전체 크기에 비례하도록 하는 것입니다. grid cell 전체 크기로 나누어주면 모델이 상대적인 크기에 대해 학습하게 됩니다.

두 번째 항은 객체를 예측하도록 할당되지 않은 grid cell의 bounding box loss입니다. 그렇다면 객체를 예측하지 않아도 되는 grid cell에 대해서 loss를 구해주는 이유는 무엇일까요? 바로 해당 grid cell이 객체를 예측하지 않도록 학습시키기 위함입니다. 이는 오직 객체를 예측하도록 할당 받은 grid cell만이 객체를 예측하도록 학습시킨다는 것을 의미합니다. 객체를 예측하도록 할당받지 않은 grid cell이 넓은 크기의 anchor box를 예측하는 것은 불필요한 일입니다. 그럼에도 not responsible for한 grid cell을 학습에 포함시키는 이유는 모델이 불필요한 시도를 하지 않게끔 유도하기 위함입니다. 즉, responsible for한 grid cell이 정확하게 객체를 detect하는 것만큼이나 not responsible for한 grid cell이 객체를 detect하지 않도록 완전히 배제하는 과정도 필요한 것입니다. 이를 위해 객체를 detect하지 못하도록 anchor box의 위치와 범위를 grid cell과 같게 줄여버리는 방법이 있습니다. 이는 anchor box의 $x_{ij}^{anchor\_center}$, $y_{ij}^{anchor\_center}$ 위치를 grid cell의 정가운데가 되도록, anchor box의 $w_{ij}^{anchor\_default}$, $h_{ij}^{anchor\_default}$가 grid cell과 같아지도록 학습시키면 됩니다.

#### Confidence loss







-

---

# YOLOv3

$$\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(x_i-\hat{x}_i\right)^2 + \left(y_i-\hat{y}_i\right)^2\right]+\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(\sqrt{w_i}-\sqrt{\hat{w}_i}\right)^2 + \left(\sqrt{h_i}-\sqrt{\hat{h}_i}\right)^2\right]$$ 
$$+\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left(C_i-\hat{C}_i\right)^2+\lambda_{noobj}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{noobj}\left(C_i-\hat{C}_i\right)^2$$  
$$+\sum_{i=0}^{S^2}\mathcal{1}_{i}^{obj}\sum_{c\in classes}\left(p_i\left(c\right)-\hat{p}_i\left(c\right)\right)$$